# Introdução

Este documento mostra um exemplo simples de chatbot com base no modelo "Saco de Palavras" que pega como entrada uma frase de Geometria Euclidiana Plana e diz se a frase é verdadeira ou falsa.

Este é um bloco de texto. As explicações gerais serão dadas em blocos como este. O bloco abaixo é um bloco de código Python. Nele, as linhas com o sinal # são comentários usados para explicar partes específicas do código.

Ao passar o mouse sobre um bloco de código como este abaixo, você verá um símbolo de "play" no lado esquerdo, que serve para excutar/rodar o script naquele bloco de código.

Para mais detalhes, como sempre, leia a dissertação disponível em

De maneira geral, o que o script abaixo faz é:


1.   Carrega as bibliotecas necessárias (já instaladas no Google Colab).

2.   Define as linhas (chamadas de "documentos") de treinamento. O saco de palavras será treinado com estas frases. Você pode alterar a lista de frases. A máquina, a princípio, não sabe diferenciar os significados do número 4 da palavra "quatro" -- e as frases que você coloca aqui tem influência neste aprendizado.

3.   Informa, na ordem, se as frases são verdeiras ou falsas.

4.   Define, treina e salva o modelo treinado.

Se você clicar "play" no código abaixo, ele vai seguir os 4 passos acima e você termina com um arquivo contendo o modelo treinado. Você não verá o arquivo, ele fica no Google Colab.



In [ ]:
!pip install keras

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten, Conv1D, MaxPooling1D

import numpy as np

# Coloque aqui as frases para treinamento, dentro de aspas simples e
# separadas por vírgula, dentro dos colchetes.
docs = [ 'Um triângulo tem 3 lados. ',
         'Um quadrilátero tem 4 lados. ',
         'Um triângulo pode ser escaleno, isósceles ou equilátero',
         'Um triângulo escaleno tem 0 lados iguais',
         'Um triângulo isósceles tem 2 lado iguais',
         'um triângulo equilátero tem 3 lados iguais',
         'Um quadrado é um quadrilátero',
         'Um quadrado é um losango',
         'Um quadrado é um retângulo',
         'Um retângulo tem 4 ângulos retos',
         'Um triângulo é um quadrilátero',
         'Um quadrilátero é um triângulo',
         'Um quadrado é um triângulo',
         'Um retângulo é um triângulo',
         'Um losango é um triângulo',
         'Um quadrado tem 0 lados iguais',
         'Um triângulo tem 2 ângulos retos',
         'Um quadrado tem 0 ângulos retos',
         'Um triângulo isósceles é escaleno',
         'Um triângulo isósceles é equilátero']

# Cada número aqui corresponde a uma frase acima, na ordem.
# O número 1 significa que a frase é verdadeira, o número 0 significa que é falsa.
labels = [1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0]

# Mude as linhas a seguir apenas se souber o que está fazendo. Veja mais detalhes
# na dissertação
labels = np.array(labels)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(docs)
sequences = tokenizer.texts_to_sequences(docs)
padded = pad_sequences(sequences, padding='post', maxlen=5)

# Definindo o modelo
vocab_size = len(tokenizer.word_index) + 1
embed_dim = 8
model = Sequential()
model.add(Embedding(vocab_size, embed_dim, input_length=5))
model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compilando e treinando o modelo em 50 épocas (fases de treinamento).
# Você pode aumentar o número de épocas para tentar melhorar a qualidade das
# respostas. Isso tem custo computacional maior e portanto o arquivo com o
# modelo treinado demorará mais para ficar pronto.
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(padded, labels, epochs=50, verbose=0)
model.save_weights("saved_weights.h5")

# Este comando salva o modelo no arquivo "trained_model.keras" aqui no Google Colab
model.save("trained_model.keras")


# Testando o modelo treinado
Agora que que o modelo está pronto, basta carregá-lo para testar. O script a seguir executa os seguintes passos:

1.   Carrega o arquivo do modelo treinado.

2.   Entra com uma frase sobre Geometria Euclidiana a ser analisada pelo modelo.

3.   Depois da frase de entrada ser colocada num padrão reconhecido pelo modelo, o script salva, na variável "saida" um valor correspondente à saída do modelo para esta frase.

4.   O script então compara a variável "saida" com a variável "limite". Se o valor de "saida" for maior que o valor de "limite", a frase será classificada como verdadeira e se for menor, será considerada falsa.

Se você clicar para rodar o código abaixo, o modelo treinado no código anterior vai classificar a frase "Um triângulo pode ser quadrado". Mude a frase e verfifique as respostas do modelo.

Como este script foi feito para servir de exemplo para a dissertação, não houve preocupação com otimização. Com isso, o modelo funciona, mas as respostas, no estágio atual, não conseguem um alto grau de acerto. A otimização deste modelo é um trabalho posterior interessante caso alguma pessoa se interesse em continuar. Além do número de épocas maior, mais frases de treinamento (preferencialmente sem expandir muito o vocabulário) e alterações nas configurações de pad_sequences, embed_dim e Conv1D podem criar modelos melhores.



In [ ]:
from keras.models import load_model

# carregando o modelo para ser usado
loaded_model = load_model("trained_model.keras")

# esta variável é a entrada, modifique-a à vontade para testar
input_text = "Um triângulo pode ser quadrado"
sequences = tokenizer.texts_to_sequences([input_text])
padded_sequences = pad_sequences(sequences, maxlen=5)

# O modelo agora vai analisar a frase de entrada e retornar um número
# que corresponde a essa análise
saida = loaded_model.predict(padded_sequences)

# agora o modelo vai interpretar o valor de saída, de acordo com o limite fornecido
limite = 0.5
if saida >= limite:
    print('"', input_text, '"'," é uma afirmação verdadeira: ", saida)
else:
    print('"', input_text, '"'," é uma afirmação falsa: ", saida)

1/1 [==============================] - 0s 125ms/step
" Um triângulo pode ser quadrado "  é uma afirmação falsa:  [[0.46942347]]


Mais alguns exemplos de resultados:

" Um quadrilátero tem 5 lados  "  é uma afirmação verdadeira:  [[0.5022047]]

" Um quadrilátero tem 0 lados "  é uma afirmação falsa:  [[0.47834444]]

" Um quadrilátero tem 2 lados "  é uma afirmação verdadeira:  [[0.50960064]]

" Um quadrilátero tem 10 lados "  é uma afirmação verdadeira:  [[0.5022047]]

" Um triângulo pode ser quadrado "  é uma afirmação falsa:  [[0.46942347]]
